导入需要使用的包

In [11]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

使用CUDA设备，并配置超参数。

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

创建sample文件夹。

In [13]:
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

载入MNIST数据集：

In [14]:
dataset = torchvision.datasets.MNIST(root='../../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

创建一个变分自编码网络([VAE](https://www.cnblogs.com/huangshiyu13/p/6209016.html))，VAE的机构为：

<div align=center><img src="https://images2015.cnblogs.com/blog/798706/201612/798706-20161222110521495-412598726.png" alt="" width="663" height="255"></div>

由于模型有重构的loss还有训练mu和std的loss,所以`forward`函数的返回是有3个的。

In [18]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    

创建一个模型实例。

In [19]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

训练情况：

In [21]:
# 训练主循环
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # 前向计算，模型的输出是最后一层，以及编码的mu和std
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # 重构误差计算
        # KL误差参考http://yunjey47.tistory.com/43 或者VAE论文的附录B
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # 反向传播
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # 保存采样图像
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # 保存生成图像
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

Epoch[1/15], Step [100/469], Reconst Loss: 11171.4082, KL Div: 3155.0068
Epoch[1/15], Step [200/469], Reconst Loss: 11525.8750, KL Div: 3125.1724
Epoch[1/15], Step [300/469], Reconst Loss: 11003.8467, KL Div: 3157.7600
Epoch[1/15], Step [400/469], Reconst Loss: 10961.0156, KL Div: 3120.3181
Epoch[2/15], Step [100/469], Reconst Loss: 10916.7617, KL Div: 3129.7771
Epoch[2/15], Step [200/469], Reconst Loss: 11275.9414, KL Div: 3250.2090
Epoch[2/15], Step [300/469], Reconst Loss: 10692.9727, KL Div: 3192.1187
Epoch[2/15], Step [400/469], Reconst Loss: 11021.7539, KL Div: 3186.9519
Epoch[3/15], Step [100/469], Reconst Loss: 10616.5264, KL Div: 3076.5391
Epoch[3/15], Step [200/469], Reconst Loss: 10710.6719, KL Div: 3238.9351
Epoch[3/15], Step [300/469], Reconst Loss: 10225.2607, KL Div: 3118.9282
Epoch[3/15], Step [400/469], Reconst Loss: 10488.7637, KL Div: 3151.3682
Epoch[4/15], Step [100/469], Reconst Loss: 10261.7764, KL Div: 3143.8379
Epoch[4/15], Step [200/469], Reconst Loss: 10625.12